In [25]:
import pandas as pd
import re

In [26]:
dir = "D:\\IoT\\Data\\hh101"

df =pd.read_csv(dir + "/" + "hh101.ann.features.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321428 entries, 0 to 321427
Data columns (total 37 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   lastSensorEventHours      321428 non-null  float64
 1   lastSensorEventSeconds    321428 non-null  float64
 2   lastSensorDayOfWeek       321428 non-null  float64
 3   windowDuration            321428 non-null  float64
 4   timeSinceLastSensorEvent  321428 non-null  float64
 5   prevDominantSensor1       321428 non-null  float64
 6   prevDominantSensor2       321428 non-null  float64
 7   lastSensorID              321428 non-null  float64
 8   lastSensorLocation        321428 non-null  float64
 9   lastMotionLocation        321428 non-null  float64
 10  complexity                321428 non-null  float64
 11  activityChange            321428 non-null  float64
 12  areaTransitions           321428 non-null  float64
 13  numDistinctSensors        321428 non-null  f

In [27]:
#Détecte des anomalies simple
def anomaly_detection(date, sensor, translate01, translate02, message, sensorType) :
    #Anomalies au niveau de la porte d'entrée   
    if int(date[11:13]) >= 0 and int(date[11:13]) <= 6 and (message == "OPEN" or message == "ON") and translate01 == "OutsideDoor":
        print("Anomaly : " , date, sensor, translate01, translate02, message, sensorType)

    #Anomalies de température
    if re.match("T\D*",sensor) and (int(message) > 40 or int(message) < 15) :
        print("Anomaly : " , date, sensor, translate01, translate02, message, sensorType)


In [28]:
#make parser for dataset <Date> <Time> <Sensor> <Translate01> <Translate02> <Message> <SensorType> <
def parse(line):
    line = line.split("\t")
    date = line[0]
    sensor = line[1]
    translate01 = line[2]
    translate02 = line[3]
    message = line[4]
    sensorType = line[5]
    anomaly_detection(date, sensor, translate01, translate02, message, sensorType)
    return date, sensor, translate01, translate02, message, sensorType
    


In [29]:
#read h101.rawdata.txt line by line
file =dir + "/" + "hh101.rawdata.txt"

data= pd.DataFrame(columns=["Date", "Sensor", "Translate01", "Translate02", "Message", "SensorType"])

with open(file) as f:
    for line in f.readlines(10000):
        #print(line)
        date, sensor, translate01, translate02, message, sensorType = parse(line)
        data.loc[len(data)] = [date, sensor, translate01, translate02, message, sensorType]


#convert date to datetime




NameError: name 're' is not defined

In [ ]:
data["Date"] = pd.to_datetime(data["Date"])

In [ ]:
def fen_gen(data,win_size,step_size):
    win_start = 0
    for i in range(0,len(data),step_size):
        win_end = win_start + win_size
        if win_end > len(data):
            break
        else:
            yield data[win_start:win_end]
            win_start += step_size 



    

In [ ]:
MotionSensorlist = data[data["SensorType"]=="Control4-Motion\n"]["Sensor"].value_counts().index.tolist()
MotionSensorlist += data[data["SensorType"]=="Control4-MotionArea\n"]["Sensor"].value_counts().index.tolist()
LightSensorlist = data[data["SensorType"]=="Control4-LightSensor\n"]["Sensor"].value_counts().index.tolist()
DoorSensorlist = data[data["SensorType"]=="Control4-Door\n"]["Sensor"].value_counts().index.tolist()
TempSensorlist = data[data["SensorType"]=="Control4-Door\n"]["Sensor"].value_counts().index.tolist()

def compute_features(fen,last_features) :
    
    #heure 
    last_date = fen["Date"].iloc[-1]
    last_hour = last_date.hour + last_date.minute/60
    duration = (fen["Date"].iloc[-1] - fen["Date"].iloc[0]).seconds

    #jour de la semaine
    week_day = last_date.weekday()
    features = [last_date,last_hour, week_day, duration]
    #motion sensor
    
    i=len(features)
    for m in MotionSensorlist:
        if fen[fen["Sensor"]==m].shape[0] > 0:
            if fen[fen["Sensor"]==m].iloc[-1]["Message"] == "ON":
                features.append(1)
            else:
                features.append(-1)
        else:
            features.append(last_features[i])
        i+=1

    

    for d in DoorSensorlist:
        if fen[fen["Sensor"]==m].shape[0] > 0:
            if fen[fen["Sensor"]==m].iloc[-1]["Message"] == "OPEN":
                features.append(1)
            else:
                features.append(-1)
        else:
            features.append(last_features[i])
        i+=1
    
    
    for l in LightSensorlist:
        if fen[fen["Sensor"]==l].shape[0] > 0:
            features.append(int(fen[fen["Sensor"]==l].iloc[-1]["Message"])/100.)
        else:
            features.append(last_features[i])
        i+=1
    
    for t in TempSensorlist:
        if fen[fen["Sensor"]==l].shape[0] > 0:
            features.append((fen[fen["Sensor"]==l].iloc[-1]["Message"])/100.)
        else:
            features.append(last_features[i])
        i+=1
    
    
    
    
    return features



In [ ]:
gen = fen_gen(data,30,15)

compute_features(next(gen),[0 for i in range(100)])

[Timestamp('2012-07-18 12:54:46.507035'),
 12.9,
 2,
 36001,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0.47,
 0.45,
 0.29,
 0.49,
 0.47,
 0.45,
 0.43,
 0.36,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
#print Sensors for sensorType= Contrl4-motion
data["Sensor"].value_counts().index

Index(['LS001', 'D002', 'M001', 'T101', 'T102', 'T103', 'T104', 'T105', 'M006',
       'M007', 'M003', 'M004', 'M005', 'M011', 'MA013', 'M008', 'M002', 'M009',
       'M012', 'MA015', 'M010', 'MA016', 'MA014', 'BATP014', 'LS005', 'LS014',
       'BATP005', 'BATP013', 'BATP015', 'LS013', 'BATP006', 'LS015', 'LS007',
       'BATP007', 'BATP004', 'LS006', 'D001', 'LS004', 'BATP008', 'BATP102',
       'BATP103', 'D003', 'LS010', 'BATP010', 'LS011', 'BATP011', 'LS008',
       'LS009', 'BATP003', 'BATP009', 'LS012', 'BATP012', 'BATP001', 'LS002',
       'BATP002', 'BATP016', 'LS003', 'LS016'],
      dtype='object')

In [ ]:
data[data["SensorType"]=="Control4-Door\n"]["Sensor"].value_counts().index.tolist()


['D002', 'D001', 'D003']

In [ ]:
# faire un modele de detection d'anomalie
#model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)


